<a href="https://colab.research.google.com/github/nicolasrondan/cv-um-2021/blob/main/trabajo-final/Face_Detection_Test_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/computer-vision-um/cv-um-2021/trabajo-final/

In [ ]:
import cv2 
import numpy as np
from glob import glob
from enum import Enum
import os
import sklearn 
import sklearn.neighbors
import matplotlib.pyplot as plt
import pickle
from evaluation import evaluate_detector, precision_and_recall, interpolated_average_precision
import sys
from image_utils import non_max_suppression
from tensorflow.keras.applications import MobileNetV2
import math
from skimage import feature
from tqdm import tqdm

In [ ]:
test_files_dir = './test_files/data/face_detection/te_raw_images/'

### Feature Extractor Code 

In [ ]:
class FeatureExtractors(Enum):
		MiniImage = 1
		HOG = 2
		LBP = 3

def extract_features(method, img):
	'''Switch between Feature extraction Methods'''

	image_representation = []

	if method == FeatureExtractors.MiniImage:
		image_representation = extract_mini_image_features(img)
	elif method == FeatureExtractors.HOG:
		image_representation = extract_hog_features(img)
	elif method == FeatureExtractors.LBP:
		image_representation = extract_lbp_features(img)	
	
	return image_representation

def extract_mini_image_features(img,resize_size=(64,64)):
	resized_image = cv2.resize(img,resize_size)
	image_representation = resized_image.reshape(resize_size[0]*resize_size[1])
	return image_representation
  
def extract_lbp_features(img):
  return []

def extract_hog_features(img):
  return []


   

In [ ]:
def sliding_window(image, window_size, scale, stride):
    [image_rows, image_cols] = image.shape;
    window_rows = window_size[0];
    window_cols = window_size[1];

    patches = np.zeros((window_rows, window_cols,5));
    bbox_locations = np.zeros((5,4))
    r = np.random.randint(0,image_rows-window_rows,5); # Sample top left position
    c = np.random.randint(0,image_cols-window_cols,5);
    for i in range(0,5):
        patches[:,:,i] = image[r[i]:r[i]+window_rows, c[i]:c[i]+window_cols];
        bbox_locations[i,:] = [r[i],c[i],window_rows,window_cols]; # top-left y,x, height, width


    return patches, bbox_locations

In [ ]:
def show_image_with_bbox(image,bboxes,draw_GT=True):
    GT = [82,91,166,175]
    if draw_GT:
        cv2.rectangle(image, (GT[0],GT[1]), (GT[2],GT[3]), (0, 0, 255), 2)

    for bbox in bboxes:
        if len(bbox) == 4:   
            top_left = (int(bbox[0]),int(bbox[1]))
            bottom_right = (int(bbox[0])+ int(bbox[2]),int(bbox[1])+int(bbox[3]))
            cv2.rectangle(image, top_left, bottom_right, (255, 0, 0), 2)

    plt.imshow(image[...,::-1])
    plt.axis('off')
    plt.show()

### Load Classifier

In [ ]:
# IF using sklearn
classifier = pickle.load(open('./face_detector','rb'))

### Evaluation

In [ ]:
total_true_positives = []
total_real_positives = []
total_positive_predictions = []
window_size = [64, 64]
folders = sorted(glob(test_files_dir + '/*'))
for subject_folder in tqdm(folders,total=len(folders)):
    for img in sorted(glob(subject_folder + '/*.jpg')):
        bgr_image = cv2.imread(img)
        gray_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2GRAY)
        patches, bbox_locations = sliding_window(gray_image,window_size,1,32)
        ## You need to extract features for every patch (same features you used for training the classifier)
        patches_feature_representation = []
        for i in range(patches.shape[2]):
            patch_representation = extract_features(FeatureExtractors.HOG, patches[:,:,i])
            patches_feature_representation.append(patch_representation)
        patches_feature_representation = np.asarray(patches_feature_representation)
        ## Get score for each sliding window patch
        scores = classifier.predict_proba(patches_feature_representation)
        ## Positive Face Probabilities
        face_probabilities = scores[:,1]
        #[labels, acc, prob] = predict([],patches_feature_representation, clasifier)
        ## Positive Face Probabilities
        #face_probabilities = np.asarray(prob)
        #face_probabilities = face_probabilities.T[0]
        
        [ detected_true_positives, image_real_positives, detected_faces ] = evaluate_detector( bbox_locations, face_probabilities);
        total_true_positives.append(detected_true_positives)
        total_real_positives.append(image_real_positives)
        total_positive_predictions.append(detected_faces)
        
total_true_positives = np.asarray(total_true_positives)
total_real_positives = np.asarray(total_real_positives)
total_positive_predictions = np.asarray(total_positive_predictions)

In [ ]:
precision, recall = precision_and_recall(total_true_positives, total_real_positives,total_positive_predictions)

In [ ]:
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.xlim(0,1.1)
plt.ylim(0,1.1)

In [ ]:
ap = interpolated_average_precision(recall,precision)
print('Detection Average Precision is {}'.format(ap))

#### VISUALIZE RESULTS #### 

In [ ]:
window_size = [64, 64]
predictions = []
threshold_p = 0.5
overlap_threshold = 0.5
folders = sorted(glob(test_files_dir + '/*'))
for subject_folder in tqdm(folders,total=len(folders)):
    for img_file in sorted(glob(subject_folder + '/*.jpg')):
      bgr_image = cv2.imread(img_file)
      gray_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2GRAY)
      patches, bbox_locations = sliding_window(gray_image,window_size,1,32)

      ## You need to extract features for every patch (same features you used for training the classifier)
      patches_feature_representation = []
      for i in range(patches.shape[2]):
          patch_representation = extract_features(FeatureExtractors.HOG, patches[:,:,i])
          patches_feature_representation.append(patch_representation)
      patches_feature_representation = np.asarray(patches_feature_representation)
      ## Get prediction label for each sliding window patch
      labels = classifier.predict(patches_feature_representation)
      ## Get score for each sliding window patch
      scores = classifier.predict_proba(patches_feature_representation)
      ## Positive Face Probabilities
      face_probabilities = scores[:,1]
      face_bboxes = bbox_locations[face_probabilities>threshold_p]
      face_bboxes_probabilites = face_probabilities[face_probabilities>threshold_p]
      # Do non max suppression and select strongest probability box
      [selected_bbox, selected_score] = non_max_suppression(face_bboxes,face_bboxes_probabilites,0.3)
      show_image_with_bbox(bgr_image, selected_bbox)
      print(selected_bbox)
